##### ATPA 2.6 - Missing and Extreme Values

In [ ]:
# Chunk 1: Prepares the flight data set for a permutation test

import pandas as pd
import numpy as np
import random

flights = pd.read_csv("flights.csv")

# transforms the time variables to minutes from midnight. 
flights = flights.assign(dep_time= flights.dep_time-40*np.floor(flights.dep_time/100), sched_dep_time = flights.sched_dep_time-40*np.floor(flights.sched_dep_time/100))

In [ ]:
# Chunk 2: Performs a permutation test for the dep_time variable in the flights data set

# Find the test statistic for the data
is_missing = flights.dep_time.isnull()
Test_stat = flights.sched_dep_time.loc[is_missing].mean() - flights.sched_dep_time.loc[-is_missing].mean()
Test_stat

# Do a for loop to reorder the data, find the R values, and create a distribution to compare against the test statistic. The permutations are done on the variable that does not have missing value. That way the is_missing vector can be reused.
random.seed(1234)
nrun = 1000
R_vals = [0 for i in range(nrun)]
ind = np.arange(len(flights))
for i in range(nrun):
  np.random.shuffle(ind)
  flights["temp_sched"] = flights.sched_dep_time[ind].reset_index().iloc[:,1]
  R_vals[i] =  flights.temp_sched[is_missing==True].mean() - flights.temp_sched.loc[is_missing==False].mean()

# Create a confidence interval
CI = np.quantile(R_vals,[.025,.975])
CI 

# Check condition
Test_stat < CI[1] and Test_stat > CI[0]

In [ ]:
# Chunk 2A: Perform the permutation test using the normal approximation

import seaborn as sns
import matplotlib.pyplot as plt

n = flights.shape[0]
m = is_missing.sum()
v = flights.sched_dep_time.var()
var_R = n*v/m/(n-m)
CI2 = 1.96 * np.sqrt(var_R)
CI2
np.abs(Test_stat) < CI2


# The following are some additional checks on the approximation
# Calculate the variance of the sample of R-values and compare to the variance based on all permutations
np.var(R_vals)
var_R

# Make a simple histogram of the R-values
plt.hist(R_vals)
plt.show()
plt.cla()
# In this case the approximation worked well

In [ ]:
# Chunk 3: Creates the production data set

# Create Data Set
production = pd.DataFrame({"Produced": [145,212,137,187,166],"Employees": [6,8,6,7,7],"Available_Machines":[19,24,np.nan,20,18],"Hours_Open":[10,9,8,9,6]})
production

In [ ]:
# Chunk 4: Mean imputation

# using SingleImputer from the autoimpute package
# Warnings produced, if any, can be ignored

from autoimpute.imputations import SingleImputer, MultipleImputer

imp = SingleImputer(strategy={"Available_Machines":'mean'})
production_mean = imp.fit_transform(production)
production_mean

In [ ]:
# Chunk 5: Regression imputation

# using SingleImputer from the autoimpute package

imp = MultipleImputer(n=1,return_list=True,strategy={"Available_Machines":'least squares'})
# you can set n to a higher value if there is randomness in the imputation procedure and you want to see multiple versions, however, you do not need to 
# return_list=True is there so we can see the output
reg_imp = imp.fit_transform(production.drop(columns=["Produced"]))[0][1] # The output creates a nested structure and [0][1] helps us get the data frame
reg_imp

In [ ]:
# Chunk 5-optional: Do the above imputation manually

# Perform the regression
from sklearn.linear_model import LinearRegression
production_clean = production.loc[~pd.isna(production.Available_Machines),:]

X = production_clean.iloc[:,[1,3]].values.reshape(-1,2)
Y = production_clean.iloc[:,2].values.reshape(-1,1)
reg = LinearRegression().fit(X,Y)
reg.predict(np.array([[6,8]]))

In [ ]:
# Chunk 6: K Nearest Neighbors imputation

from sklearn.impute import KNNImputer

colnames = production.columns
imputer = KNNImputer(n_neighbors=2, weights="uniform")
pd.DataFrame(imputer.fit_transform(production.drop(columns=["Produced"])), columns = colnames[1:4])

In [ ]:
# Chunk 7: Imputing both categorical and numeric variables

production["Manager"] = ["On","Off","On","Off",np.nan]

imputer = SingleImputer(strategy={"Manager":"categorical"})
data_imputed = imputer.fit_transform(production)
data_imputed

# when using regression methods with mixed data types you need to binarize the categorical variables
newprod = pd.get_dummies(production.drop(columns=["Produced"]),drop_first=True)
imputer = MultipleImputer(return_list=True,n=1,strategy={"Available_Machines":"least squares","Manager_On":"binary logistic"})
data_imputed = imputer.fit_transform(newprod)[0][1]
data_imputed

# Note that the two imputation methods do not produce the same results for Manager

In [ ]:
# Chunk 8: Exercise 2.6.1 Check for Understanding Questions

import seaborn as sns
import matplotlib.pyplot as plt
# Read in autombile data set
Auto_Names = ["symboling", "normalized_losses", "make", "fuel_type","aspiration", "num_doors", "body_style", "drive_wheels","engine_location", "wheel_base", "length", "width", "height", "curb_weight", "engine_type","num_cylinders", "engine_size", "fuel_system", "bore", "stroke", "compression_ratio", "horsepower", "peak_rpm", "city_mpg", "highway_mpg", "price"]

auto = pd.read_csv("automobile.csv",names=Auto_Names)
cols_to_keep = ["normalized_losses","num_doors","body_style","curb_weight","engine_type","bore","city_mpg","price"]
auto = auto[cols_to_keep]

# Replace ?'s with NAs
auto = auto.replace("?",np.nan)
# Change these three columns from character to numeric
auto = auto.assign(normalized_losses = [float(x) for x in auto.normalized_losses],bore = [float(x) for x in auto.bore],price = [float(x) for x in auto.price])

# Examine the normalized_loss variable for missingness at random.
# 1. Check a box plot for curb_weight and city_mpg when normalized_loss is and isn't missing

# 2. Perform a permutation test for the normalized_loss variable using curb_weight as the variable without missing data.

# 3. Explain the missingness of bore

# 4. Perform mean imputation for the missing values of price

# 5. Perform regression imputation for price

# 6. Perform imputation for both price and num_doors

In [ ]:
# Chunk 9A: Exercise 2.6.1: Question 1

import seaborn as sns
import matplotlib.pyplot as plt
# Read in autombile data set
Auto_Names = ["symboling", "normalized_losses", "make", "fuel_type","aspiration", "num_doors", "body_style", "drive_wheels","engine_location", "wheel_base", "length", "width", "height", "curb_weight", "engine_type","num_cylinders", "engine_size", "fuel_system", "bore", "stroke", "compression_ratio", "horsepower", "peak_rpm", "city_mpg", "highway_mpg", "price"]

auto = pd.read_csv("automobile.csv",names=Auto_Names)
cols_to_keep = ["normalized_losses","num_doors","body_style","curb_weight","engine_type","bore","city_mpg","price"]
auto = auto[cols_to_keep]

# Replace ?'s with NAs
auto = auto.replace("?",np.nan)

# Convert numeric variables to numeric type
auto = auto.assign(normalized_losses = [float(x) for x in auto.normalized_losses],bore = [float(x) for x in auto.bore],price = [float(x) for x in auto.price])

# Examine the normalized_losses variable for missingness at random.
# 1. Check a box plot for curb_weight and city_mpg when normalized losses is and isn't missing
auto["missing"] = auto.normalized_losses.isnull()
sns.boxplot(x = "missing",y="curb_weight",data=auto)
plt.show()
plt.cla()
sns.boxplot(x = "missing",y="city_mpg",data=auto)
plt.show()
plt.cla()

# It seems as if there may be a relationship between missingness and both curb_weight city_mpg.

In [ ]:
# Chunk 9B: Exercise 2.6.1: Question 2

# 2. Perform a permutation test for the normalized_loss variable using curb_weight as the variable without missing data.
Test_stat = auto.curb_weight[auto.missing].mean()-auto.curb_weight[-auto.missing].mean()
Test_stat

# Do a for loop to reorder the data, find the R values, and create a distribution to compare against the test statistic
random.seed(1234)
nrun = 1000
R_vals = [0 for i in range(nrun)]
ind = np.arange(len(auto))
for i in range(nrun):
  np.random.shuffle(ind)
  auto["temp_curb_weight"] = auto.curb_weight[ind].reset_index().iloc[:,1]
  R_vals[i] = auto.temp_curb_weight[auto.missing].mean()-auto.temp_curb_weight[-auto.missing].mean()

# Create a confidence interval
CI = np.quantile(R_vals,[.025,.975])
CI 

# Check condition
Test_stat < CI[1] and Test_stat > CI[0]
# normalized_losses are not missing at random
auto = auto.drop(columns=["missing"])

In [ ]:
# Chunk 9C: Exercise 2.6.1: Question 3

# 3. Explain the missingness of bore with relationship to engine_type

# Look at the four records where bore is missing
pd.set_option('max_columns', None) # This will display all the columns
auto.loc[pd.isna(auto.bore),:]

# All have engine_type = rotor. Now check that this is the only time rotor appears
auto.loc[auto.engine_type == "rotor",:]

# When bore is missing, those are also the only incidences of the engine_type = rotor.
# At this point an investigation of this relationship might be in order. We will elect to delete these records, but that will imply any model we build will not be able to make predictions for cars with that engine_type.

auto = auto.loc[~pd.isna(auto.bore),:]

In [ ]:
# Chunk 9D: Exercise 2.6.1: Question 4

# 4. Perform mean imputation for the missing values of price. 

price_missing = auto.price.isnull()

imp = SingleImputer(strategy={"price":'mean'})
auto_mean = imp.fit_transform(auto)
auto_mean.loc[price_missing,"price"]

In [ ]:
# Chunk 9E: Exercise 2.6.1: Question 5

# 5. Perform regression imputation for the price

auto_dummies = pd.get_dummies(auto.drop(columns=["normalized_losses"]),drop_first=True)

imp = MultipleImputer(n=1,return_list=True,strategy={"price":'least squares'})
auto_reg = imp.fit_transform(auto_dummies)[0][1]
auto_reg.loc[price_missing,"price"]

In [ ]:
# Chunk 9F: Exercise 2.6.1: Question 6

# 6. Perform imputation for both price and num_doors.

doors_missing = auto.num_doors.isnull()

imp = MultipleImputer(n=1,return_list=True,strategy={"price":'least squares',"num_doors_two":'binary logistic'})
auto_reg_2 = imp.fit_transform(auto_dummies)[0][1]
auto_reg_2.loc[doors_missing,"num_doors_two"]
auto_reg_2.loc[price_missing,"price"]

In [ ]:
# Chunk 10: Identifying outliers

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

hotel_bookings = pd.read_csv("hotel_bookings.csv")

# full data boxplot
sns.boxplot(y = "stays_in_week_nights",data=hotel_bookings)
plt.show()
plt.cla()

# full data histogram
sns.displot(hotel_bookings.stays_in_week_nights,binwidth=1)
plt.show()
plt.close()

In [ ]:
# Chunk 11: Looking for outliers

# Try cutoffs of 3, 5, and 10, 

cutoff = 5

hotel_trimmed = hotel_bookings.assign(zscores = (hotel_bookings.stays_in_week_nights-hotel_bookings.stays_in_week_nights.mean())/hotel_bookings.stays_in_week_nights.std())
hotel_trimmed = hotel_trimmed.loc[hotel_trimmed.zscores < cutoff,:]
100*(1 - len(hotel_trimmed.index)/len(hotel_bookings.index))
sns.displot(hotel_trimmed.stays_in_week_nights,binwidth=1)
plt.title("Trimmed at Zscores above " + str(cutoff))
plt.show()
plt.close()

In [ ]:
# ChunkS 12 - 15 Various methods to deal with outliers.
# Chunk 12: 1. Log transform
Because there are 0's, the transform is log(x+1)

hotel_log = hotel_bookings.assign(logstay = np.log(hotel_bookings.stays_in_week_nights+1))
sns.displot(hotel_log.logstay,binwidth=.5)
plt.title("Log Transformation")
plt.show()
plt.close()

In [ ]:
# Chunk 13: 2. Removing outliers. This is the same as in # Chunk 11. The cutoff in this example is a z-score of 5.

cutoff = 5
hotel_trimmed = hotel_bookings.assign(zscores = (hotel_bookings.stays_in_week_nights-hotel_bookings.stays_in_week_nights.mean())/hotel_bookings.stays_in_week_nights.std())
hotel_trimmed = hotel_trimmed.loc[hotel_trimmed.zscores < cutoff,:]
sns.displot(hotel_trimmed.stays_in_week_nights,binwidth=1)
plt.title("Trimmed at Zscores above " + str(cutoff))
plt.show()
plt.close()

In [ ]:
# Chunk 14: 3. Change the values. In this case we can cap all observations above 6 as being equal to 6. 

cap =  6
hotel_capped = hotel_bookings
hotel_capped.stays_in_week_nights = hotel_capped.stays_in_week_nights.clip(upper=cap)
sns.displot(hotel_capped.stays_in_week_nights,binwidth=1)
plt.title("Capped at "+str(cap))
plt.show()
plt.close()

In [ ]:
# Chunk 15: 4. Replace variable with percentiles.  

hotel_bookings = pd.read_csv("hotel_bookings.csv")

hotel_bookings["perc_stay"] = hotel_bookings.stays_in_week_nights.rank(pct=True)

plt.plot(hotel_bookings.stays_in_week_nights,hotel_bookings.perc_stay,"o")
plt.xlabel("Weeknight Stays",y="Percentiles")
plt.title("Percentile Transform by Raw Data")
plt.show() 
plt.close()

In [ ]:
# Chunk 16: Use DBSCAN to find outliers based on several observations. Change eps and minPts to affect the algorithm. The number of outliers are the number of noise points and given a cluster value of -1. 

from sklearn.cluster import DBSCAN
hotel_small = hotel_bookings.loc[:,["stays_in_week_nights","stays_in_weekend_nights","adults"]]
hotel_small = hotel_small.loc[0:9999,:]

eps = 3
minPts = 10
X = hotel_small.to_numpy()
clustering = DBSCAN(eps = eps, min_samples = minPts).fit(X)
cluster = pd.to_numeric(clustering.labels_)
cluster = pd.DataFrame(cluster, columns=["cluster"])
hotel_small2 = pd.concat([hotel_small,cluster],axis=1)
hotel_small2.query("cluster < 0")